In [13]:
import numpy as np
import pandas as pd
pd.options.display.float_format = "{:,.2f}".format
from tabulate import tabulate

In [14]:
health_df = pd.read_csv('data/data_new.csv')
def FNR(a, b):
    return ((a == False) & (b == True)).sum() / (b == True).sum()

def FPR(a, b):
    return ((a == True) & (b == False)).sum() / (b == False).sum()

In [5]:
np.cov(cost_screen_in, health_screen_in)

array([[0.24733246, 0.05398149],
       [0.05398149, 0.21621068]])

In [6]:
np.cov(health_df['cost_t'], health_df['gagne_sum_t'])

array([[3.23637265e+08, 1.23956691e+04],
       [1.23956691e+04, 3.77326037e+00]])

In [12]:
outcome_df['enrolled'].sum()

452

In [19]:
acceptance_cuttoff = .55 # Physician referral cutoff
proxy = 'total_cost'


cost_screen_in = health_df['cost_t'] > np.quantile(health_df['cost_t'], acceptance_cuttoff)
health_screen_in = health_df['gagne_sum_t'] > np.quantile(health_df['gagne_sum_t'], acceptance_cuttoff)
cost_avoid_screen_in = health_df['cost_avoidable_t'] > np.quantile(health_df['cost_avoidable_t'], acceptance_cuttoff)

outcome_df = pd.concat([cost_screen_in, cost_avoid_screen_in, health_screen_in, health_df['program_enrolled_t']], axis=1)
outcome_df.rename(columns={
            'cost_t': 'total_cost',
             'cost_avoidable_t': 'avoidable_cost',
             'gagne_sum_t': 'health',
             'program_enrolled_t': 'enrolled'
            }, inplace=True)

control = outcome_df[outcome_df['enrolled'] == 0]
treatment = outcome_df[outcome_df['enrolled'] == 1]
obs = outcome_df

obs_fpr = FPR(obs[proxy], obs["health"])
obs_fnr = FNR(obs[proxy], obs["health"])

control_fpr = FPR(control[proxy], control["health"])
control_fnr = FNR(control[proxy], control["health"])

treatment_fpr = FPR(treatment[proxy], treatment["health"])
treatment_fnr = FNR(treatment[proxy], treatment["health"])

obs_conditions = health_df['gagne_sum_t'].mean()
control_conditions = health_df['gagne_sum_t'][health_df['program_enrolled_t'] == 0].mean()
treatment_conditions = health_df['gagne_sum_t'][health_df['program_enrolled_t'] == 1].mean()

obs_cost = health_df['cost_t'].mean()
control_cost = health_df['cost_t'][health_df['program_enrolled_t'] == 0].mean()
treatment_cost = health_df['cost_t'][health_df['program_enrolled_t'] == 1].mean()

proxy_metrics = pd.DataFrame({
    'Population': ['Full population', 'Unenrolled', 'Enrolled'],
    'FPR': [obs_fpr, control_fpr, treatment_fpr],
    'FNR': [obs_fnr, control_fnr, treatment_fnr],
    'N': [health_df.shape[0],(health_df['program_enrolled_t'] == 0).sum(), (health_df['program_enrolled_t'] == 1).sum()]
})

obs_alpha, treatment_alpha = obs_fpr, treatment_fpr
obs_beta, treatment_beta = obs_fnr, treatment_fnr


# Threshold based on treatment-conditional rate
t_treatment = (1 - treatment_beta + treatment_alpha)/2

# Threshold based on observational rate (ignoring treatment effects and framing as a prediction policy problem)
t_obs = (1 - obs_beta + obs_alpha)/2

# Threshold based on no correction
t_treatment_uncorrected = .5 


print('Treatment corrected', t_treatment)
print('Observational', t_obs)
print('Treatment uncorrected', t_treatment_uncorrected)


print(tabulate(proxy_metrics, headers = 'keys', tablefmt = 'latex', floatfmt=".2f"))


Treatment corrected 0.7572591587516961
Observational 0.4941998890514145
Treatment uncorrected 0.5
\begin{tabular}{rlrrr}
\hline
    & Population      &   FPR &   FNR &     N \\
\hline
  0 & Full population &  0.37 &  0.38 & 48784 \\
  1 & Unenrolled      &  0.37 &  0.39 & 48332 \\
  2 & Enrolled        &  0.64 &  0.13 &   452 \\
\hline
\end{tabular}


In [23]:
proxy_metrics.drop_index()

AttributeError: 'DataFrame' object has no attribute 'drop_index'

In [ ]:
0.0158

torch.isnan(input

    Population         FPR    FNR
--  ---------------  -----  -----
 0  Full population   0.02   0.72
 1  Unenrolled        0.02   0.73
 2  Enrolled          0.14   0.62


In [152]:
obs_fpr = FPR(obs["avoidable_cost"], obs["health"])
obs_fnr = FNR(obs["avoidable_cost"], obs["health"])

control_fpr = FPR(control["avoidable_cost"], control["health"])
control_fnr = FNR(control["avoidable_cost"], control["health"])

treatment_fpr = FPR(treatment["avoidable_cost"], treatment["health"])
treatment_fnr = FNR(treatment["avoidable_cost"], treatment["health"])

print(f'Observational: FPR {obs_fpr:.2}, FNR {obs_fnr:.2}')
print(f'Treatment: FPR {treatment_fpr:.2}, FNR {treatment_fnr:.2}')
print(f'Control: FPR {control_fpr:.2}, FNR {control_fnr:.2}')

Decide: 

Observational: FPR 0.19, FNR 0.57
Treatment: FPR 0.39, FNR 0.26
Control: FPR 0.19, FNR 0.58


In [191]:
health_df['gagne_sum_t'][health_df['program_enrolled_t'] == 1].mean()

4.632743362831858

In [194]:
health_df['cost_avoidable_t'][health_df['program_enrolled_t'] == 0].mean()

2328.0476702805595

In [195]:
health_df['cost_avoidable_t'][health_df['program_enrolled_t'] == 1].mean()

13841.371681415929

In [196]:
health_df['cost_t'][health_df['program_enrolled_t'] == 1].mean()

27163.938053097347

In [158]:
treatment["avoidable_cost"].mean()

0.6858407079646017

In [159]:
control["avoidable_cost"].mean()

0.26044856409831996

In [94]:
obs_beta = ((obs['cost_t'] == False) & (obs['gagne_sum_t'] == True) ).sum() / (obs['gagne_sum_t'] == True).sum()

obs_alpha = ((obs['cost_t'] == True) & (obs['gagne_sum_t'] == False) ).sum() / (obs['gagne_sum_t'] == False).sum()



In [56]:
obs_alpha

0.4125299760191847

In [45]:
((cost_screen_in == True) & (health_screen_in == False) ).sum()

0

In [39]:
obs_beta

0.4890907927570542

In [44]:
obs_alpha

0.0

In [35]:
obs_fpr

0.4890907927570542

In [36]:
obs_alpha

0.0

In [23]:
(cost_screen_in & health_screen_in).sum()/cost_screen_in.sum()

1.0

In [9]:
cost_cuttoff

2800.0

In [6]:
screen_in_cost

2800.0

In [12]:

sub_feats = health_df[['program_enrolled_t', 'cost_t', 'gagne_sum_t', 'race', 'cost_avoidable_t']]
sub_feats.groupby(['program_enrolled_t', 'race']).mean()


cost_t  gagne_sum_t  cost_avoidable_t
program_enrolled_t race                                              
0                  black   8889.187716     2.003816       3245.829548
                   white   7295.904644     1.236452       2210.123981
1                  black  33545.569620     5.658228      16426.582278
                   white  25812.332440     4.415550      13293.833780